### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Loading the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [6]:
### Groq model

from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020943923490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020943A0FFD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French:"),
    HumanMessage(content="I love programming in Python!")
]

result = model.invoke(messages)

In [11]:
result

AIMessage(content="J'adore programmer en Python\xa0! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 23, 'total_tokens': 35, 'completion_time': 0.021818182, 'prompt_time': 0.002148745, 'queue_time': 0.245166085, 'total_time': 0.023966927}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--86748b42-2a64-4bfe-896f-bc447439d14c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 12, 'total_tokens': 35})

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

"J'adore programmer en Python\xa0! \n"

In [12]:
### Using LCEL - chain the components
chain = model | parser

chain.invoke(messages)

"J'adore programmer en Python ! \n"

In [15]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}: "

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ],
)

In [18]:
result = prompt.invoke({"language": "French", "text": "I love programming in Python!"})

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following into French: ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming in Python!', additional_kwargs={}, response_metadata={})]

In [ ]:
### Chaining together components with LCEL

chain = prompt|model|parser

chain.invoke({"language": "French", "text": "I love programming in Python!"})

"J'adore programmer en Python ! \n\n\nLet me know if you'd like me to translate anything else!\n"